In [1]:
from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
import os

/home/eugene/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
fn = "data/3/data.TIF"
fn_mod = "data/3/data_mod.TIF"
fn_res = "data/3/data_res.TIF"

In [3]:
os.remove(fn_mod) if os.path.exists(fn_mod) else None
os.remove(fn_res) if os.path.exists(fn_res) else None
ds = gdal.Open(fn, gdal.GA_ReadOnly)

In [4]:
print("Projection: ", ds.GetProjection())  # get projection
print("Geotransform: ", ds.GetGeoTransform())
print("Columns:", ds.RasterXSize)  # number of columns
print("Rows:", ds.RasterYSize)  # number of rows
print("Band count:", ds.RasterCount)  # number of bands
print("Size : ", os.path.getsize(fn))

Projection:  PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32648"]]
Geotransform:  (246717.33307071, 0.3, 0.0, 1989720.3061312002, 0.0, -0.3)
Columns: 16384
Rows: 12344
Band count: 4
Size :  1618052106


In [5]:
def copy_geo_transform_and_projection(from_dataset, to_dataset):
    to_dataset.SetGeoTransform(from_dataset.GetGeoTransform())
    to_dataset.SetProjection(from_dataset.GetProjection())

In [6]:
n = ds.RasterCount
data_type = ds.GetRasterBand(1).DataType
metadata = ds.GetMetadata('IMAGE_STRUCTURE')
compress = False

if 'COMPRESSION' in metadata:
    compress = True
    print(metadata['COMPRESSION'])
    options = ["COMPRESS=" + metadata['COMPRESSION']]
# compress = True
# options = ["COMPRESS=DEFLATE"]

In [17]:
data_diff = {}
stored_data = {} #key is diff value; value is 2d array consists of [[1,250,251], [1,120, 251], [2,100,599], [3,522,120], ..]
# first index = the band
# second index = the X
# third index = the Y

In [8]:
# # TODO
# # MODIFY THIS ACCORDING TO WHERE IS THE 2nd, 3rd and 4th band stored

# for i in range(n):
#     # if i == 0: 
#     #     continue
#     data_del[i+1] = ds.GetRasterBand(i+1).ReadAsArray()

In [18]:
# TODO
# MODIFY THIS ACCORDING TO WHERE IS THE 2nd, 3rd and 4th band stored
first_band_data = ds.GetRasterBand(1).ReadAsArray()

for i in range(n):
    if i == 0: 
        continue
    data_arr = ds.GetRasterBand(i+1).ReadAsArray()
    data_diff[i+1] = np.subtract(first_band_data, data_arr)
    

In [14]:
print(data_diff)

{2: array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16), 3: array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16), 4: array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)}


In [20]:
for key, arr in data_diff.items():
    for (i,j), value in np.ndenumerate(arr):
        if value == 0 : #skip if 0
            continue
        stored_data.setdefault(value, []).append([key, i, j])

In [14]:
driver = gdal.GetDriverByName('GTiff')

if compress:
    ds_mod = driver.Create(fn_mod, ds.RasterXSize, ds.RasterYSize, 1, data_type, options=options)
else:
    ds_mod = driver.Create(fn_mod, ds.RasterXSize, ds.RasterYSize, 1, data_type)

copy_geo_transform_and_projection(ds, ds_mod)

In [15]:
ds_mod.GetRasterBand(1).WriteArray(data_del[1])

0

In [16]:
print("Projection: ", ds_mod.GetProjection())  # get projection
print("Geotransform: ", ds_mod.GetGeoTransform())
print("Columns:", ds_mod.RasterXSize)  # number of columns
print("Rows:", ds_mod.RasterYSize)  # number of rows
print("Band count:", ds_mod.RasterCount)  # number of bands
ds_mod = None

Projection:  PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32648"]]
Geotransform:  (246717.33307071, 0.3, 0.0, 1989720.3061312002, 0.0, -0.3)
Columns: 16384
Rows: 12344
Band count: 1


In [17]:
print("Size : ", os.path.getsize(fn_mod))

Size :  404562616


In [18]:
if compress:
    ds_res = driver.Create(fn_res, ds.RasterXSize, ds.RasterYSize, n, data_type, options=options)
else:
    ds_res = driver.Create(fn_res, ds.RasterXSize, ds.RasterYSize, n, data_type)

copy_geo_transform_and_projection(ds, ds_res)

In [20]:
# Assuming the goal is to restore the "deleted" band to its original position (n_del)
for i in range(n):
    ds_res.GetRasterBand(i+1).WriteArray(data_del[i+1])


In [21]:
print("Projection: ", ds_res.GetProjection())  # get projection
print("Geotransform: ", ds_res.GetGeoTransform())
print("Columns:", ds_res.RasterXSize)  # number of columns
print("Rows:", ds_res.RasterYSize)  # number of rows
print("Band count:", ds_res.RasterCount)  # number of bands

Projection:  PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32648"]]
Geotransform:  (246717.33307071, 0.3, 0.0, 1989720.3061312002, 0.0, -0.3)
Columns: 16384
Rows: 12344
Band count: 4


In [22]:
ds = None
ds_res = None

In [23]:
print("Size : ", os.path.getsize(fn_res))

Size :  1618051914
